In [1]:
# Courtesy: https://github.com/mchablani/deep-learning/blob/master/sentiment-rnn/Sentiment_RNN.ipynb
# Sentiment Analysis with an LSTM-RNN::
# Six classes:
# 1. Positive
# 2. Neutral
# 3. Depressed
# 4. Negative but not depressed
# 5. Ambiguous
# 6. Incomplete

In [2]:
import numpy as np
import tensorflow as tf
#from tensorflow.keras.callbacks import TensorBoard
from timeit import default_timer as timer
#import time

In [3]:
with open('../tweet_5000_all_sentiments_six_classes_text.txt', 'r', encoding="utf8") as f:
    reviews = f.read()
with open('../tweet_5000_all_sentiments_six_classes_labels.txt', 'r', encoding="utf8") as f:
    labels_org = f.read()

In [4]:
labels_org[:1002]

'neutral\npositive\nneutral\npositive\npositive\nambiguous\npositive\npositive\nambiguous\npositive\npositive\npositive\nambiguous\ndepressive\npositive\npositive\ndepressive\npositive\npositive\nincomplete\npositive\npositive\ndepressive\nincomplete\npositive\npositive\nneutral\ndepressive\nneutral\nneutral\nneutral\npositive\npositive\npositive\npositive\ndepressive\npositive\nincomplete\npositive\npositive\ndepressive\npositive\nincomplete\npositive\npositive\npositive\nneutral\npositive\ndepressive\nambiguous\npositive\npositive\npositive\npositive\nneutral\npositive\nneutral\nneutral\npositive\npositive\npositive\nincomplete\nneutral\npositive\nsim_negative\nambiguous\npositive\nambiguous\npositive\npositive\ndepressive\npositive\nambiguous\npositive\nneutral\npositive\npositive\npositive\npositive\ndepressive\nneutral\npositive\npositive\npositive\nsim_negative\npositive\npositive\npositive\nincomplete\ndepressive\nneutral\npositive\ndepressive\npositive\nneutral\ndepressive\npos

In [5]:
reviews[:1000]

'জনগণের ভোট চুরি করে ক্ষমতায় আসা ওই নিয়ত আমাদের নাই আওয়ামী লীগ নেত্রী শেখ হাসিনা এভাবে নৌকার পক্ষে ভোটে ডাকাতি করছেন লজ্জা লাগেনা ছিঃ বাসদ নেত্রী ড মনীষা চক্রবর্তী । \nসবথেকে বড় যুদ্ধ হলো নিজের মনের সাথে যুদ্ধ আবেগ\nতোমাকে মনে পড়বে যখনিই জ্যোস্না হাসে তোমাকে মনে পড়বে যখনিই আকাশ ভেঙ্গে বর্ষা কাঁদে পাগল\nরোজ আকাশে একটা তারা জলমল করতো হাজার মেঘলা এমনকি ঘন বর্ষাতেও আমি খুঁজে পাইতাম । শুভ সকাল \nআল্লাহ মহাপরাক্রমশালী এবং সবার ওপর পরিপূর্ণরূপে বিজয়ী হওয়া সত্ত্বেও নিজের সৃষ্টির প্রতি তিনি দয়াবান ও ক্ষমাশীল । দুষ্কর্মকারীদের শাস্তি দেয়ার পুরো ক্ষমতা তাঁর আছে । কিন্তু যারা লজ্জিত হয়ে দুষ্কর্ম পরিত্যাগ করে এবং ক্ষমা প্রার্থনা করে তাদের সাথে তিনি ক্ষমাশীলতার আচরণ করে থাকেন । \nআদরের ডাক যদি মুছে , এই নাও কিছু ঘুম পাড়ানি গান আলগোছে , বোঝনা এটুকু শিলালিপি , মন রে ব্যাথার আদরে অবুঝ আঙ্গুল রাখলাম \nনিষিদ্ধ পল্লীতে নিষিদ্ধ মানুষের বসবাস থাকলেও সেখানকার খরিদ্দাররা একদম বিশুদ্ধ পল্লীর হয়\nসবাই কেমন আছেন আশা করি ভালো আছেন\nএনআৰচি ৰ ওপৰত মোৰ মন্তব্য । \nযত ধৈর্য্য তত বেশি সফলতা । সমস্যা হলো আ

In [6]:
# Data preprocessing::

from string import punctuation
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

In [7]:
all_text[:2000]

'জনগণের ভোট চুরি করে ক্ষমতায় আসা ওই নিয়ত আমাদের নাই আওয়ামী লীগ নেত্রী শেখ হাসিনা এভাবে নৌকার পক্ষে ভোটে ডাকাতি করছেন লজ্জা লাগেনা ছিঃ বাসদ নেত্রী ড মনীষা চক্রবর্তী ।  সবথেকে বড় যুদ্ধ হলো নিজের মনের সাথে যুদ্ধ আবেগ তোমাকে মনে পড়বে যখনিই জ্যোস্না হাসে তোমাকে মনে পড়বে যখনিই আকাশ ভেঙ্গে বর্ষা কাঁদে পাগল রোজ আকাশে একটা তারা জলমল করতো হাজার মেঘলা এমনকি ঘন বর্ষাতেও আমি খুঁজে পাইতাম । শুভ সকাল  আল্লাহ মহাপরাক্রমশালী এবং সবার ওপর পরিপূর্ণরূপে বিজয়ী হওয়া সত্ত্বেও নিজের সৃষ্টির প্রতি তিনি দয়াবান ও ক্ষমাশীল । দুষ্কর্মকারীদের শাস্তি দেয়ার পুরো ক্ষমতা তাঁর আছে । কিন্তু যারা লজ্জিত হয়ে দুষ্কর্ম পরিত্যাগ করে এবং ক্ষমা প্রার্থনা করে তাদের সাথে তিনি ক্ষমাশীলতার আচরণ করে থাকেন ।  আদরের ডাক যদি মুছে  এই নাও কিছু ঘুম পাড়ানি গান আলগোছে  বোঝনা এটুকু শিলালিপি  মন রে ব্যাথার আদরে অবুঝ আঙ্গুল রাখলাম  নিষিদ্ধ পল্লীতে নিষিদ্ধ মানুষের বসবাস থাকলেও সেখানকার খরিদ্দাররা একদম বিশুদ্ধ পল্লীর হয় সবাই কেমন আছেন আশা করি ভালো আছেন এনআৰচি ৰ ওপৰত মোৰ মন্তব্য ।  যত ধৈর্য্য তত বেশি সফলতা । সমস্যা হলো আমাদের ধৈয্যধ

In [8]:
words[:100]

['জনগণের',
 'ভোট',
 'চুরি',
 'করে',
 'ক্ষমতায়',
 'আসা',
 'ওই',
 'নিয়ত',
 'আমাদের',
 'নাই',
 'আওয়ামী',
 'লীগ',
 'নেত্রী',
 'শেখ',
 'হাসিনা',
 'এভাবে',
 'নৌকার',
 'পক্ষে',
 'ভোটে',
 'ডাকাতি',
 'করছেন',
 'লজ্জা',
 'লাগেনা',
 'ছিঃ',
 'বাসদ',
 'নেত্রী',
 'ড',
 'মনীষা',
 'চক্রবর্তী',
 '।',
 'সবথেকে',
 'বড়',
 'যুদ্ধ',
 'হলো',
 'নিজের',
 'মনের',
 'সাথে',
 'যুদ্ধ',
 'আবেগ',
 'তোমাকে',
 'মনে',
 'পড়বে',
 'যখনিই',
 'জ্যোস্না',
 'হাসে',
 'তোমাকে',
 'মনে',
 'পড়বে',
 'যখনিই',
 'আকাশ',
 'ভেঙ্গে',
 'বর্ষা',
 'কাঁদে',
 'পাগল',
 'রোজ',
 'আকাশে',
 'একটা',
 'তারা',
 'জলমল',
 'করতো',
 'হাজার',
 'মেঘলা',
 'এমনকি',
 'ঘন',
 'বর্ষাতেও',
 'আমি',
 'খুঁজে',
 'পাইতাম',
 '।',
 'শুভ',
 'সকাল',
 'আল্লাহ',
 'মহাপরাক্রমশালী',
 'এবং',
 'সবার',
 'ওপর',
 'পরিপূর্ণরূপে',
 'বিজয়ী',
 'হওয়া',
 'সত্ত্বেও',
 'নিজের',
 'সৃষ্টির',
 'প্রতি',
 'তিনি',
 'দয়াবান',
 'ও',
 'ক্ষমাশীল',
 '।',
 'দুষ্কর্মকারীদের',
 'শাস্তি',
 'দেয়ার',
 'পুরো',
 'ক্ষমতা',
 'তাঁর',
 'আছে',
 '।',
 'কিন্তু',
 'যারা',
 'লজ্জিত',
 'হয়ে']

In [9]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

reviews_ints = []
for each in reviews:
    reviews_ints.append([vocab_to_int[word] for word in each.split()])

In [10]:
print(len(reviews_ints))

5001


In [11]:
# Encoding the labels::

# Convert labels to 1s and 0s for 'positive' and 'negative'
# print(labels_org)
labels = np.array([1 if l == "positive" else 0 for l in labels_org.split()])
# print(labels)
print(len(labels))

5000


In [12]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 61


In [13]:
# Filter out that review with 0 length
reviews_ints = [r[0:200] for r in reviews_ints if len(r) > 0]

In [14]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 61


In [15]:
seq_len = 200
features = np.zeros((len(reviews_ints), seq_len), dtype=int)
# print(features[:10,:100])
for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
features[:10,:100]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

In [16]:
print(len(features))
print(type(features))
print(features[41])
print(len(features[41]))
print(reviews_ints[41])
print(len(reviews_ints[41]))

5000
<class 'numpy.ndarray'>
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0  206 3951   69  535 3952 3953 2092
 2718 3954   26 2719 1286  729 3955   91    6 39

In [17]:
split_frac = 0.8

split_index = int(split_frac * len(features))

train_x, val_x = features[:split_index], features[split_index:] 
train_y, val_y = labels[:split_index], labels[split_index:]

split_frac = 0.5
split_index = int(split_frac * len(val_x))

val_x, test_x = val_x[:split_index], val_x[split_index:]
val_y, test_y = val_y[:split_index], val_y[split_index:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))
print("label set: \t\t{}".format(train_y.shape), 
      "\nValidation label set: \t{}".format(val_y.shape),
      "\nTest label set: \t\t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		(4000, 200) 
Validation set: 	(500, 200) 
Test set: 		(500, 200)
label set: 		(4000,) 
Validation label set: 	(500,) 
Test label set: 		(500,)


In [18]:
# Build the graph::

lstm_size = 256
#lstm_layers = 2
lstm_layers = 2
#batch_size = 1000
batch_size = 50
learning_rate = 0.01

In [19]:
n_words = len(vocab_to_int) + 1 # Add 1 for 0 added to vocab

# Create the graph object
tf.reset_default_graph()
with tf.name_scope('inputs'):
    inputs_ = tf.placeholder(tf.int32, [None, None], name="inputs")
    labels_ = tf.placeholder(tf.int32, [None, None], name="labels")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")

In [20]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with tf.name_scope("Embeddings"):
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [21]:
def lstm_cell():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
    # Add dropout to the cell
    return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

with tf.name_scope("RNN_layers"):
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [22]:
with tf.name_scope("RNN_forward"):
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)

In [23]:
# Output::

with tf.name_scope('predictions'):
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    tf.summary.histogram('predictions', predictions)
with tf.name_scope('cost'):
    cost = tf.losses.mean_squared_error(labels_, predictions)
    tf.summary.scalar('cost', cost)

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

merged = tf.summary.merge_all()

In [24]:
# Validation accuracy::

with tf.name_scope('validation'):
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [25]:
# Batching::

def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [26]:
# Training::

epochs = 10

# with graph.as_default():
saver = tf.train.Saver()
start = timer()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./logs/tb/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/tb/test', sess.graph)
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            summary, loss, state, _ = sess.run([merged, cost, final_state, optimizer], feed_dict=feed)
#             loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)

            train_writer.add_summary(summary, iteration)
        
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
#                     batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    summary, batch_acc, val_state = sess.run([merged, accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
            test_writer.add_summary(summary, iteration)
            saver.save(sess, "checkpoints/tweet_5000_all_sentiments_six_classes.ckpt")
#            tensorboard = TensorBoard(log_dir="logs/tweet_5000_all_sentiments_six_classes-{}".format(int(time.time())))
    saver.save(sess, "checkpoints/tweet_5000_all_sentiments_six_classes.ckpt")
    
duration = timer() - start
print('Time elasped =',duration,'sec(s)')

Epoch: 0/10 Iteration: 5 Train loss: 0.440
Epoch: 0/10 Iteration: 10 Train loss: 0.327
Epoch: 0/10 Iteration: 15 Train loss: 0.293
Epoch: 0/10 Iteration: 20 Train loss: 0.281
Epoch: 0/10 Iteration: 25 Train loss: 0.253
Val acc: 0.450
Epoch: 0/10 Iteration: 30 Train loss: 0.235
Epoch: 0/10 Iteration: 35 Train loss: 0.270
Epoch: 0/10 Iteration: 40 Train loss: 0.243
Epoch: 0/10 Iteration: 45 Train loss: 0.262
Epoch: 0/10 Iteration: 50 Train loss: 0.243
Val acc: 0.450
Epoch: 0/10 Iteration: 55 Train loss: 0.304
Epoch: 0/10 Iteration: 60 Train loss: 0.269
Epoch: 0/10 Iteration: 65 Train loss: 0.244
Epoch: 0/10 Iteration: 70 Train loss: 0.346
Epoch: 0/10 Iteration: 75 Train loss: 0.276
Val acc: 0.550
Epoch: 0/10 Iteration: 80 Train loss: 0.250
Epoch: 1/10 Iteration: 85 Train loss: 0.262
Epoch: 1/10 Iteration: 90 Train loss: 0.249
Epoch: 1/10 Iteration: 95 Train loss: 0.216
Epoch: 1/10 Iteration: 100 Train loss: 0.222
Val acc: 0.620
Epoch: 1/10 Iteration: 105 Train loss: 0.299
Epoch: 1/10 Ite

In [27]:
# Testing::

test_acc = []
with tf.Session() as sess:
    saver.restore(sess, "checkpoints/tweet_5000_all_sentiments_six_classes.ckpt")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints/tweet_5000_all_sentiments_six_classes.ckpt
Test accuracy: 0.808
